https://github.com/nilportugues/reinforcement-learning-1/blob/master/Code%201.%20Grid%20World/6.%20DQN/Gridworld_DQN.py

In [8]:
import random
import copy
import pylab
import random
import numpy as np
from collections import deque
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Sequential

In [31]:
class Env():
    
    metadata = {'render.modes': ['human']}
    
    def __init__(self, row, col, bingo, bingo_state, bingo_selection_count, debug = False):
        super(Env, self).__init__()
        
        self.row = row
        self.col = col       
        self.action_space = spaces.Discrete(self.row*self.col)
        self.current_step = 0
        self.bingo = bingo
        self.bingo_state = bingo_state
        self.bingo_selection_count = bingo_selection_count  
        #To count how many times a number is repeated for bigger -ve reward
        self.number_cut = 0
        self.strikes = 0
        self.action_space = spaces.Discrete(self.row*self.col)
        self.observation_space = spaces.Box(
          low=np.zeros(self.row*self.col).reshape(self.row,self.col), 
            high=np.ones(self.row*self.col).reshape(self.row,self.col), dtype=np.int32)
        self.debug = debug
        
    def reset(self):
               
        self.current_step = 0
        self.bingo = np.array(random.sample(range(1,self.row*self.col+1), self.row*self.col)).reshape(self.row,self.col)
        self.bingo_state = np.zeros(self.row*self.col).reshape(self.row,self.col).astype('int32')
        self.bingo_selection_count = np.zeros(self.row*self.col).reshape(self.row,self.col).astype('int32')
        bingo_selection_count = np.zeros(self.row*self.col).reshape(self.row,self.col).astype('int32')
        self.strikes = 0
        
        return self._next_observation()
    
    def _next_observation(self):
        
        current_bingo = self.bingo
        current_bingo_state = self.bingo_state
        

        return np.array(current_bingo)
  
    def _take_action(self, action):
        
        self.current_step+=1
        self.number_cut = self.bingo[math.floor(action/self.row),action%self.row]
    
        already_cut_reward = 0
        if(self.number_cut > 0):
            self.bingo[math.floor(action/self.row),(action)%self.row] = 0
            self.bingo_state[math.floor(action/self.row),action%self.row] = 1
            self.bingo_selection_count[math.floor(action/self.row),action%self.row] = 1
        else:
            self.bingo_selection_count[math.floor(action/self.row),action%self.row] += 1
            already_cut_reward = -self.bingo_selection_count[math.floor(action/self.row),action%self.row] -2           
        
        #For cuts
        cuts = 0
        for i in range(self.row):
            if(self.bingo_state[i,:].sum() == self.row):
                cuts+=1
        for i in range(self.col):
            if(self.bingo_state[:,i].sum() == self.col):
                cuts+=1
        diag1_sum = 0
        diag2_sum = 0
        
        for i in range(self.row):
            diag1_sum+= self.bingo_state[i,i]
            diag2_sum+= self.bingo_state[i,self.row-i-1]
            
        if(diag1_sum == self.row):            
            cuts+=1
        if(diag2_sum == self.row):            
            cuts+=1
    
        #bingo_dict = {0: "No_cut", 1: "B", 2: "BI", 3: "BIN", 4: "BING", 5: "BINGO", 6: "BINGO", 7: "BINGO"}
        
        reward_for_multiple_cuts = 0
        cuts_in_one_go = cuts - self.strikes
        if(cuts_in_one_go > 1 and cuts <6):
            reward_for_multiple_cuts = cuts_in_one_go
        self.strikes = cuts
        
        total_add_reward = already_cut_reward + reward_for_multiple_cuts
                        
        return cuts, total_add_reward
    
    def step(self,action):
        
        reward = -1
        
        total_cuts,add_reward = self._take_action(action)
        reward+= add_reward
        
        if (total_cuts >= self.row):
            done = True
        else:
            done = False
        
        return self._next_observation(), reward, done, {}
    
    def render(self, mode='train', close=False):
        # Render the environment to the screen
        
        if(mode=='train'):
            train_list.append([self.bingo, self.bingo_state, self.bingo_selection_count, "BINGO", self.strikes])
        else:
            test_list.append([self.bingo, self.bingo_state, self.bingo_selection_count, "BINGO", self.strikes])
            
        return
    
    

In [32]:
row = 5
col = 5
bingo = np.array(random.sample(range(1,row*col+1), row*col)).reshape(row,col)
bingo_state = np.zeros(row*col).reshape(row,col).astype('int32')
bingo_selection_count = np.zeros(row*col).reshape(row,col).astype('int32')

bingo_env = Env(row, col, bingo, bingo_state, bingo_selection_count, debug = True)

In [ ]:


EPISODES = 1000

# this is DQN Agent
# it uses Neural Network to approximate q function
# and replay memory & target q network
class DQNAgent:
    def __init__(self,row,col):
        # if you want to see Cartpole learning, then change to True
        self.render = False

        # actions which agent can do
        self.action_space = range(0,row*col)
        # get size of state and action
        self.action_size = len(self.action_space)
        self.state_size = (row,col)
        self.discount_factor = 0.99
        self.learning_rate = 0.01

        self.epsilon = 1.  # exploration
        self.epsilon_decay = .9999
        self.epsilon_min = 0.01
        self.batch_size = 32
        self.train_start = 100

        # create replay memory using deque
        self.memory = deque(maxlen=10000)
        self.model = self.build_model()
        self.target_model = self.build_model()
        # copy the model to target model
        # --> initialize the target model so that the parameters of model & target model to be same
        self.update_target_model()

    # approximate Q function using Neural Network
    # state is input and Q Value of each action is output of network
    def build_model(self):
        model = Sequential()
        model.add(Dense(20, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(20, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(self.action_size, activation='linear', kernel_initializer='he_uniform'))
        model.summary()
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    # after some time interval update the target model to be same with model
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # get action from model using epsilon-greedy policy
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            # The agent acts randomly
            return random.randrange(self.action_size)
        else:
            # Predict the reward value based on the given state
            state = np.float32(state)
            q_values = self.model.predict(state)
            return np.argmax(q_values[0])

    # save sample <s,a,r,s'> to the replay memory
    def replay_memory(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    # pick samples randomly from replay memory (with batch_size)
    def train_replay(self):
        if len(self.memory) < self.train_start:
            return
        batch_size = min(self.batch_size, len(self.memory))
        mini_batch = random.sample(self.memory, batch_size)

        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.action_size))

        for i in range(batch_size):
            state, action, reward, next_state, done = mini_batch[i]
            reward = np.int32(reward)
            state = np.int32(state)
            next_state = np.int32(next_state)
            target = self.model.predict(state)[0]

            # like Q Learning, get maximum Q value at s'
            # But from target model
            if done:
                target[action] = reward
            else:
                target = reward + self.discount_factor * \
                                  np.amax(self.model.predict(next_state)[0])

            update_input[i] = state
            update_target[i] = target

        # make minibatch which includes target q value and predicted q value
        # and do the model fit!
        self.model.fit(update_input, update_target, batch_size=batch_size, epochs=1, verbose=0)

    # load the saved model
    def load_model(self, name):
        self.model.load_weights(name)

    # save the model which is under training
    def save_model(self, name):
        self.model.save_weights(name)

In [1]:
if __name__ == "__main__":
    # maze game
    # env = Maze()
    env = ()
    agent = DQNAgent()

    global_step = 0
    # agent.load("same_vel_episode2 : 1000")
    scores, episodes = [], []

    for e in range(EPISODES):
        done = False
        score = 0
        state = env.reset()
        state = np.reshape(state, [1, 22])

        while(done==False):
            # fresh env
            if agent.render:
                env.render()
            global_step += 1

            # get action for the current state and go one step in environment
            action = agent.get_action(state)
            next_state, reward, done = env.step(action)
            next_state = np.reshape(next_state, [1, 22])

            agent.replay_memory(state, action, reward, next_state, done)
            # every time step we do training
            agent.train_replay()
            score += reward

            state = copy.deepcopy(next_state)
            print("reward:", reward, "  done:", done, "  time_step:", global_step, "  epsilon:", agent.epsilon)

            # every 100 time steps update the target model to be same with model
            if global_step % 100 == 0:
                agent.update_target_model()

            if done:
                scores.append(score)
                episodes.append(e)
                pylab.plot(episodes, scores, 'b')
                pylab.savefig("./save_graph/10by10.png")
                print("episode:", e, "  score:", score, "  memory length:", len(agent.memory),
                      "  epsilon:", agent.epsilon)

        if e % 100 == 0:
            pass
            agent.save_model("./save_model/10by10")

    # end of game
    print('game over')
    env.destroy()

NameError: name 'Env' is not defined